In [471]:
from redminelib import Redmine
from datetime import datetime
from datetime import timezone
from datetime import date
import pandas as pd
import getpass
import numpy as np
import os
from dotenv import load_dotenv

from pandas.tseries.offsets import DateOffset

from functools import reduce

dotenv_path = os.path.join(os.path.dirname('__file__'), '.env');
load_dotenv(dotenv_path);

pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:,.2f}'.format

In [472]:
# Extrai os arquivos e gera uma base de dados mocada
arquivo_gestao_simplificada = f"{os.path.dirname(os.path.abspath('__file__'))}/database/Gestao Simplificada - SEDI.xlsx"
arquivo_niveis_simplificada = f"{os.path.dirname(os.path.abspath('__file__'))}/database/2021-11 - Contrato 13_2020 - Niveis de Complexidade Ajuste - v13.xlsx"

try:
    gestao_simplificada = {
        "colaboradores": pd.read_excel(io=arquivo_gestao_simplificada, sheet_name="Colaboradores"),
        "perfil_de_complexidade": pd.read_excel(io=arquivo_gestao_simplificada, sheet_name="Perfil-Complexidade"),
        "catalogo": pd.read_excel(io=arquivo_gestao_simplificada, sheet_name="Catalogo_SEDI"),
        "grupo_de_despesa": pd.read_excel(io=arquivo_gestao_simplificada, sheet_name="Grupo de Despesa"),
        "catalogo_redmine": pd.read_excel(io=arquivo_gestao_simplificada, sheet_name="Join_Redmine_Catalogo"),
        "complexidade": pd.read_excel(io=arquivo_niveis_simplificada, sheet_name="Soluções de TI", header=2)
    }
    
except Exception as e:
    print(f"Error: {e}")
    
# Cria a instancia para o redmine
rm = Redmine(
    os.environ["URL_REDMINE"], 
    username=os.environ["USER_REDMINE"], 
    password=os.environ["PASSWORD_REDMINE"]
)

/home/gustavo/workspace/SEFAZ/memora/automacao/venv/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [473]:
#ids_colaboradores = list(colaboradores['id_redmine'].dropna())

ano = input("Ano: ")
mes = input("Mês: ")
dia = input("Dia: ")


Ano: 2022
Mês: 01
Dia: 01


In [474]:
# Converte DF de colaboradores em um array de objetos para montar a extração de tempo lançado por 'colaborador'
colaboradores_array = gestao_simplificada['colaboradores'].to_dict("records");

async def extrair_tempo_lancado(colaboradores=[]):
    # Extrai as informações de tempo lançado por 'id do colaborador' e 'data'
    # E criado um dicionário de colaboradores com seus respectivos tempos lançados pelo filtro
    tempo_lancado = {}
    for colab in colaboradores:
        temp=[]
        _filtro = rm.time_entry.filter(user_id=colab["id_redmine"]).values()
        for detales in _filtro:
            temp.append(detales)

        tempo_lancado[colab["id_redmine"]] = pd.DataFrame.from_records([t for t in temp]);
        
    return tempo_lancado;

async def extrair_tarefas_colaboradores(colaboradores=[]):
    # Extrai informaçãos das tarefas do por 'id do colaborador' e 'data de criação'
    tarefas={}
    for colab in colaboradores: 
        temp=[]
        #_filtro = rm.issue.filter(user_id=colab["id_redmine"], created_on=date(int(ano), int(mes), int(dia))).values()
        _filtro = rm.issue.filter(assigned_to_id=colab["id_redmine"], status_id="*").values()
        for detales in _filtro:
            temp.append(detales)
        tarefas[colab["id_redmine"]] = pd.DataFrame.from_records([t for t in temp]);

    return tarefas

In [475]:
# Extrai o tempo lançado de todos os colaboradores
tempo_lancado = await extrair_tempo_lancado(colaboradores_array)

In [477]:
# Renomeia alguans colunas para mergear com a tabela de tarefas e não sobreescrever os colunas
for colaborador in tempo_lancado:
    tempo_lancado[colaborador].rename(columns={'id':'id_tempo_lancado',
                                'created_on':'created_on_tempo_lancado',
                                'updated_on': 'updated_on_tempo_lancado'}, inplace=True)

In [478]:
# Extrai as tarefas por colaborador
tarefas = await extrair_tarefas_colaboradores(colaboradores_array)

# Renomeia alguans colunas para mergear com a tabela de tarefas e não sobreescrever os colunas
for colaborador in tarefas:
    tarefas[colaborador].rename(columns={'id':'id_tarefas',
                                'created_on':'created_on_tarefas',
                                'updated_on': 'updated_on_tarefas'}, inplace=True)

# Converte campos de 'date string' em 'datetieme'
async def normaliza_datas(tarefas={}, columns=[]):
    data_carga = datetime.now();
    for colaborador in tarefas: 
        for column in columns:
            try:
                tarefas[colaborador][column] = pd.to_datetime((tarefas[colaborador][column]))
                tarefas[colaborador].loc[:, 'data_carga'] = data_carga;
            except Exception as e:
                print(f"Coluna: {column} não normalizada: \n\t {e}");

In [479]:
columns_data = ['due_date', 'closed_on', 'created_on','start_date', 'updated_on']
await normaliza_datas(tarefas, columns_data)


Coluna: created_on não normalizada: 
	 'created_on'
Coluna: updated_on não normalizada: 
	 'updated_on'
Coluna: created_on não normalizada: 
	 'created_on'
Coluna: updated_on não normalizada: 
	 'updated_on'


In [480]:
parent = lambda p: '' if p==0 else p['id']
# Normaliza o DataFrame das issues por colaborador
for colaborador in tarefas:
    try:
        # Retira os dados NaN
        tarefas[colaborador] = tarefas[colaborador].dropna(
            subset = [
                'parent',
                'author',
                'project',
                'status',
                'tracker'
            ]
        )
        
        tarefas[colaborador].loc[:, 'parent'] = [parent(pai) for pai in tarefas[colaborador]['parent']];
        tarefas[colaborador].loc[:, 'author_id'] = [user['id'] for user in tarefas[colaborador]['author']];
        tarefas[colaborador].loc[:, 'author_name'] = [user['name'] for user in tarefas[colaborador]['author']];
        tarefas[colaborador].loc[:, 'project'] = [project['name'] for project in tarefas[colaborador]['project']];
        tarefas[colaborador].loc[:, 'status'] = [status['name'] for status in tarefas[colaborador]['status']];
        tarefas[colaborador].loc[:, 'Tipo'] = [tipo['name'] for tipo in tarefas[colaborador]['tracker']]
    except Exception as e:
        print(f"Erro na normalização das issues: \n\t {e}");

In [481]:
extrator_id = lambda x: None if not x else x["id"]
extrator_name = lambda x: None if not x else x["name"]

# Normaliza alguns campos dos DF de tempo lançado
for t in tempo_lancado:
    try:
        tempo_lancado[t].loc[:, "issue"] = tempo_lancado[t]["issue"].apply(extrator_id)
    except Exception as e: 
        print("Erro na normalização ca coluna issue: ", e)
    
    try:
        tempo_lancado[t].loc[:, "user_id"] = tempo_lancado[t]["user"].apply(extrator_id)
    except Exception as e:
        print("Erro na normalização do 'id' do usuário: ", e)
    
    try:
        tempo_lancado[t].loc[:, 'user_name'] = tempo_lancado[t]["user"].map(extrator_name)
    except Exception as e: 
        print("Erro na normalização do 'nome' do usuário: ", e)
        
    try:
        tempo_lancado[t].loc[:, "project_id"] = tempo_lancado[t]["project"].apply(extrator_id)
    except Exception as e:
        print("Erro na normalização do 'id projeto': ", e)
    
    try:
        tempo_lancado[t]['project_name'] = tempo_lancado[t]["project"].map(extrator_name)
    except Exception as e: 
        print("Erro na normalização do 'nome projeto': ", e)
    
    print("Dropando as colunas que já foram normalizadas ...")
    try: 
        tempo_lancado[t].drop(columns=["user"], inplace=True)
    except:
        pass
    try:
        tempo_lancado[t].drop(columns=["project"], inplace=True)
    except:
        pass
    
    print(f"Renomeando as colunas para o DataFrame do colaborador {t}")
    tempo_lancado[t].rename(columns={'user_id':'assigned_to_id',
                                 'user_name':'assigned_to_name',
                                 'hours':'HET (Real)'}, inplace=True)

Dropando as colunas que já foram normalizadas ...
Renomeando as colunas para o DataFrame do colaborador 401
Dropando as colunas que já foram normalizadas ...
Renomeando as colunas para o DataFrame do colaborador 672


In [482]:
# Mapeia os colaboradores na relação de tarefas e extrai a informação do 'Serviço (memora)'
def adiciona_catalogo_redmine(tarefas={}):
    try:
        print("Iniciando o mapeamento de campos customizados");
        def get_Catalogo_01(campos):
            servico=['0']
            servico=[servico['value'] for servico in campos if servico['id'] == 119]
            
            if not servico or servico==['']:
                return None
            return servico[0] if servico[0] else servico
        
        # TODO: Validar se o sistema modulo realmente e esse id ('120')
        def get_sistema_modulo(campos):
            mod=[' ']
            mod=[modulo['value'] for modulo in campos if modulo['id']==120]
            if mod==[''] or len(mod) == 0:
                return [' ']
            return mod

        for colaborador in tarefas:
            tarefas[colaborador] = tarefas[colaborador].dropna(subset=["custom_fields"])
            # Obtem o catalogo da memora pelo custon field
            tarefas[colaborador].loc[:,'catalogo_redmine']=tarefas[colaborador]['custom_fields'].apply(get_Catalogo_01);
            #tarefas[colaborador].loc[:,'sistema_modulo']=tarefas[colaborador]['custom_fields'].apply(get_sistema_modulo);
            print(f"Mapeado serviço memora para o colaborador: {gestao_simplificada['colaboradores'].query(f'id_redmine == {colaborador}')['Colaborador'].values.item()}");

        print("Colaboradores mapeados!");
    except Exception as e: 
        print(f"Erro no mapeamento do catalogo redmine por colaborador: \n\t{e}")

In [483]:
# chamada do mapeamento e extração do 'Serviço (memora)' por colaborador
adiciona_catalogo_redmine(tarefas)

Iniciando o mapeamento de campos customizados
Mapeado serviço memora para o colaborador: Fernando de Araujo Moraes Rodarte
Mapeado serviço memora para o colaborador: Leonardo Xavier da Silva
Colaboradores mapeados!


In [484]:
# Sumariza os dados de tempo lançado
for colaborador in tempo_lancado: 
        tempo_lancado[colaborador] = (
            tempo_lancado[colaborador].groupby(['issue', 'assigned_to_id', 'assigned_to_name']).sum().reset_index()
        )

In [485]:
# Mergea os dados de tarefas com o tempo lancado
def merge_tempo_lancado_e_tarefas(tarefas, tempo_lancado):
    aux = {}
    for colaborador in gestao_simplificada['colaboradores']["id_redmine"]:
        try:
            aux[colaborador] = pd.merge(tarefas[colaborador], tempo_lancado[colaborador], how='inner', left_on='id_tarefas', right_on='issue')
            
            aux[colaborador]['assigned_to_id'].fillna(pd.Series([extrator_id(user) for user in aux[colaborador]['assigned_to']]), inplace=True)
            aux[colaborador]['assigned_to_name'].fillna(pd.Series([extrator_name(user) for user in aux[colaborador]['assigned_to']]), inplace=True)
            aux[colaborador]['HET (Real)'].fillna(0, inplace=True)
            
            # TODO: Refatorar, não e necessário o merge uma vez que temos o author na tabela
            aux[colaborador] = pd.merge(aux[colaborador], gestao_simplificada['colaboradores']['id_redmine'], left_on='assigned_to_id', 
                           right_on='id_redmine', how='inner').drop(['id_redmine'], axis=1)
            


        except Exception as e:
            print(f"Erro no merge dos dados: \n\t{e}");
            raise Exce
    return aux;

In [486]:
tarefas[401]

,id_tarefas,project,tracker,status,priority,author,assigned_to,fixed_version,parent,subject,description,start_date,due_date,done_ratio,is_private,estimated_hours,custom_fields,created_on_tarefas,updated_on_tarefas,closed_on,data_carga,author_id,author_name,Tipo,catalogo_redmine
0,107742,Arquitetura para Aplicações Python,"{'id': 40, 'name': 'Atividade (STI - Memora)'}",Pronta p/ revisão do preposto,"{'id': 2, 'name': 'Média'}","{'id': 401, 'name': 'Fernando de Araujo Moraes...","{'id': 401, 'name': 'Fernando de Araujo Moraes...","{'id': 3838, 'name': 'N/A'}",102022,Elaborar a ATA da Reunião de Entrega de Ativid...,Elaborar a ATA da Reunião de Entrega de Ativid...,2022-02-17,2022-02-18,100,False,1.00,"[{'id': 119, 'name': 'Serviço (Memora)', 'valu...",2022-02-17T19:50:56Z,2022-02-18T13:25:17Z,2022-02-18 13:25:17+00:00,2022-02-18 11:13:33.153562,401,Fernando de Araujo Moraes Rodarte,Atividade (STI - Memora),36. - [apoio ao desenvolvimento] Participação...
1,107741,Arquitetura para Aplicações Python,"{'id': 40, 'name': 'Atividade (STI - Memora)'}",Pronta p/ revisão do preposto,"{'id': 2, 'name': 'Média'}","{'id': 401, 'name': 'Fernando de Araujo Moraes...","{'id': 401, 'name': 'Fernando de Araujo Moraes...","{'id': 3838, 'name': 'N/A'}",102022,Reunião de Entrega de Atividades,Reunião de Entrega de Atividades,2022-02-17,2022-02-17,100,False,1.00,"[{'id': 119, 'name': 'Serviço (Memora)', 'valu...",2022-02-17T19:49:10Z,2022-02-17T20:05:21Z,2022-02-17 20:05:21+00:00,2022-02-18 11:13:33.153562,401,Fernando de Araujo Moraes Rodarte,Atividade (STI - Memora),36. - [apoio ao desenvolvimento] Participação...
2,107714,LIGO - Laboratório de Inovação Goiás,"{'id': 40, 'name': 'Atividade (STI - Memora)'}",Pronta p/ revisão do preposto,"{'id': 2, 'name': 'Média'}","{'id': 401, 'name': 'Fernando de Araujo Moraes...","{'id': 401, 'name': 'Fernando de Araujo Moraes...","{'id': 3838, 'name': 'N/A'}",106253,Participação da Reunião Ordinária do Comitê de...,Participação da Reunião Ordinária do Comitê de...,2022-02-17,2022-02-17,100,False,1.00,"[{'id': 119, 'name': 'Serviço (Memora)', 'valu...",2022-02-17T17:38:06Z,2022-02-17T17:46:12Z,2022-02-17 17:46:12+00:00,2022-02-18 11:13:33.153562,401,Fernando de Araujo Moraes Rodarte,Atividade (STI - Memora),36. - [apoio ao desenvolvimento] Participação...
3,107408,TR BALANCAS GOINFRA + ECONOMIA,"{'id': 40, 'name': 'Atividade (STI - Memora)'}",Pronta p/ revisão do preposto,"{'id': 2, 'name': 'Média'}","{'id': 401, 'name': 'Fernando de Araujo Moraes...","{'id': 401, 'name': 'Fernando de Araujo Moraes...","{'id': 3838, 'name': 'N/A'}",105636,Elaborar a ATA da Reunião de Entrega de Ativid...,Elaborar a ATA da Reunião de Entrega de Ativid...,2022-02-15,2022-02-15,100,False,1.00,"[{'id': 119, 'name': 'Serviço (Memora)', 'valu...",2022-02-15T16:30:36Z,2022-02-15T19:04:39Z,2022-02-15 19:04:39+00:00,2022-02-18 11:13:33.153562,401,Fernando de Araujo Moraes Rodarte,Atividade (STI - Memora),36. - [apoio ao desenvolvimento] Participação...
4,107407,TR BALANCAS GOINFRA + ECONOMIA,"{'id': 40, 'name': 'Atividade (STI - Memora)'}",Pronta p/ revisão do preposto,"{'id': 2, 'name': 'Média'}","{'id': 401, 'name': 'Fernando de Araujo Moraes...","{'id': 401, 'name': 'Fernando de Araujo Moraes...","{'id': 3838, 'name': 'N/A'}",105636,Reunião de Entrega de Atividades e Acompanhame...,Reunião de Entrega de Atividades e Acompanhame...,2022-02-15,2022-02-15,100,False,1.00,"[{'id': 119, 'name': 'Serviço (Memora)', 'valu...",2022-02-15T16:27:42Z,2022-02-15T16:43:46Z,2022-02-15 16:43:46+00:00,2022-02-18 11:13:33.153562,401,Fernando de Araujo Moraes Rodarte,Atividade (STI - Memora),36. - [apoio ao desenvolvimento] Participação...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164,68737,Plataforma de Correção de Redações utilizando ...,"{'id': 28, 'name': 'Atividade (SEDI)'}",Aprovada (Fechada),"{'id': 2, 'name': 'Média'}","{'id': 401, 'name': 'Fernando de Araujo Moraes...","{'id': 401

In [487]:
tempo_lancado[401]

,issue,assigned_to_id,assigned_to_name,id_tempo_lancado,HET (Real),project_id
0,100782,401,Fernando de Araujo Moraes Rodarte,67981,1.00,557
1,101037,401,Fernando de Araujo Moraes Rodarte,68289,1.00,557
2,101197,401,Fernando de Araujo Moraes Rodarte,68561,1.00,557
3,102028,401,Fernando de Araujo Moraes Rodarte,69974,1.00,768
4,102036,401,Fernando de Araujo Moraes Rodarte,69900,1.00,768
...,...,...,...,...,...,...
97,107407,401,Fernando de Araujo Moraes Rodarte,76852,0.50,788
98,107408,401,Fernando de Araujo Moraes Rodarte,76887,1.00,788
99,107714,401,Fernando de Araujo Moraes Rodarte,77190,1.00,489
100,107741,401,Fernando de Araujo Moraes Rodarte,77212,1.00,768


In [488]:
tarefas_prev = merge_tempo_lancado_e_tarefas(tarefas, tempo_lancado);

In [489]:
tarefas_df = tarefas_prev.copy()

In [490]:
tarefas_df[401]

,id_tarefas,project,tracker,status,priority,author,assigned_to,fixed_version,parent,subject,description,start_date,due_date,done_ratio,is_private,estimated_hours,custom_fields,created_on_tarefas,updated_on_tarefas,closed_on,data_carga,author_id,author_name,Tipo,catalogo_redmine,issue,assigned_to_id,assigned_to_name,id_tempo_lancado,HET (Real),project_id
0,107742,Arquitetura para Aplicações Python,"{'id': 40, 'name': 'Atividade (STI - Memora)'}",Pronta p/ revisão do preposto,"{'id': 2, 'name': 'Média'}","{'id': 401, 'name': 'Fernando de Araujo Moraes...","{'id': 401, 'name': 'Fernando de Araujo Moraes...","{'id': 3838, 'name': 'N/A'}",102022,Elaborar a ATA da Reunião de Entrega de Ativid...,Elaborar a ATA da Reunião de Entrega de Ativid...,2022-02-17,2022-02-18,100,False,1.00,"[{'id': 119, 'name': 'Serviço (Memora)', 'valu...",2022-02-17T19:50:56Z,2022-02-18T13:25:17Z,2022-02-18 13:25:17+00:00,2022-02-18 11:13:33.153562,401,Fernando de Araujo Moraes Rodarte,Atividade (STI - Memora),36. - [apoio ao desenvolvimento] Participação...,107742,401,Fernando de Araujo Moraes Rodarte,77281,1.00,768
1,107741,Arquitetura para Aplicações Python,"{'id': 40, 'name': 'Atividade (STI - Memora)'}",Pronta p/ revisão do preposto,"{'id': 2, 'name': 'Média'}","{'id': 401, 'name': 'Fernando de Araujo Moraes...","{'id': 401, 'name': 'Fernando de Araujo Moraes...","{'id': 3838, 'name': 'N/A'}",102022,Reunião de Entrega de Atividades,Reunião de Entrega de Atividades,2022-02-17,2022-02-17,100,False,1.00,"[{'id': 119, 'name': 'Serviço (Memora)', 'valu...",2022-02-17T19:49:10Z,2022-02-17T20:05:21Z,2022-02-17 20:05:21+00:00,2022-02-18 11:13:33.153562,401,Fernando de Araujo Moraes Rodarte,Atividade (STI - Memora),36. - [apoio ao desenvolvimento] Participação...,107741,401,Fernando de Araujo Moraes Rodarte,77212,1.00,768
2,107714,LIGO - Laboratório de Inovação Goiás,"{'id': 40, 'name': 'Atividade (STI - Memora)'}",Pronta p/ revisão do preposto,"{'id': 2, 'name': 'Média'}","{'id': 401, 'name': 'Fernando de Araujo Moraes...","{'id': 401, 'name': 'Fernando de Araujo Moraes...","{'id': 3838, 'name': 'N/A'}",106253,Participação da Reunião Ordinária do Comitê de...,Participação da Reunião Ordinária do Comitê de...,2022-02-17,2022-02-17,100,False,1.00,"[{'id': 119, 'name': 'Serviço (Memora)', 'valu...",2022-02-17T17:38:06Z,2022-02-17T17:46:12Z,2022-02-17 17:46:12+00:00,2022-02-18 11:13:33.153562,401,Fernando de Araujo Moraes Rodarte,Atividade (STI - Memora),36. - [apoio ao desenvolvimento] Participação...,107714,401,Fernando de Araujo Moraes Rodarte,77190,1.00,489
3,107408,TR BALANCAS GOINFRA + ECONOMIA,"{'id': 40, 'name': 'Atividade (STI - Memora)'}",Pronta p/ revisão do preposto,"{'id': 2, 'name': 'Média'}","{'id': 401, 'name': 'Fernando de Araujo Moraes...","{'id': 401, 'name': 'Fernando de Araujo Moraes...","{'id': 3838, 'name': 'N/A'}",105636,Elaborar a ATA da Reunião de Entrega de Ativid...,Elaborar a ATA da Reunião de Entrega de Ativid...,2022-02-15,2022-02-15,100,False,1.00,"[{'id': 119, 'name': 'Serviço (Memora)', 'valu...",2022-02-15T16:30:36Z,2022-02-15T19:04:39Z,2022-02-15 19:04:39+00:00,2022-02-18 11:13:33.153562,401,Fernando de Araujo Moraes Rodarte,Atividade (STI - Memora),36. - [apoio ao desenvolvimento] Participação...,107408,401,Fernando de Araujo Moraes Rodarte,76887,1.00,788
4,107407,TR BALANCAS GOINFRA + ECONOMIA,"{'id': 40, 'name': 'Atividade (STI - Memora)'}",Pronta p/ revisão do preposto,"{'id': 2, 'name': 'Média'}","{'id': 401, 'name': 'Fernando de Araujo Moraes...","{'id': 401, 'name': 'Fernando de Araujo Moraes...","{'id': 3838, 'name': 'N/A'}",105636,Reunião de Entrega de Atividades e Acompanhame...,Reunião de Entrega de Atividades e Acompanhame...,2022-02-15,2022-02-15,100,False,1.00,"[{'id': 119, 'name': 'Serviço (Memora)', 'valu...",2022-02-15T16:27:42Z,2022-02-15T16:43:46Z,2022-02-15 16:43:46+00:00,2022-02-18 11:13:33.153562,401,Fernando de Araujo Moraes Rodarte,Atividade (STI - Memora),36. - [apoio ao desenvolvimento] Participação...

In [491]:
def normaliza_tarefas_merge(tarefas): 
    for colaborador in tarefas: 
        
        #tarefas[colaborador].loc[:,'dias_desde_criacao'] = None
        tarefas[colaborador]['dias_desde_criacao'] = None
        tarefas[colaborador]['dias_desde_atualizacao'] = None
        tarefas[colaborador]['dias_desde_inicio'] = None
        tarefas[colaborador]['dias_ate_prazo'] = None
        for index, valor in enumerate(tarefas[colaborador]['data_carga']):
            tarefas[colaborador].at[index, 'dias_desde_criacao'] = (
                    (tarefas[colaborador]['data_carga'][index] - pd.to_datetime(tarefas[colaborador]['created_on_tarefas'][index]).replace(tzinfo=None)).days
                )
                
                
            tarefas[colaborador].at[index,'dias_desde_atualizacao'] = (
                    (tarefas[colaborador]['data_carga'][index] - (pd.to_datetime(tarefas[colaborador]['updated_on_tarefas'][index]).replace(tzinfo=None))).days
                )
                
            tarefas[colaborador].at[index,'dias_desde_inicio'] = (
                    (tarefas[colaborador]['data_carga'][index] - tarefas[colaborador]['start_date'][index].replace(tzinfo=None)).days
                )
                
            tarefas[colaborador].at[index,'dias_ate_prazo'] = (
                    ((tarefas[colaborador]['due_date'][index].replace(tzinfo=None)) - tarefas[colaborador]['data_carga'][index]).days
                )
        

            tarefas[colaborador]['estimated_hours'].fillna(0, inplace=True)
            tarefas[colaborador].loc[:,'qtd_demandas'] = 1

In [492]:
normaliza_tarefas_merge(tarefas_df)

In [493]:
gestao_simplificada['catalogo_redmine']

,Catalogo_redmine,Codg_relacionamento
0,1.a - [análise e projeto] Concepção da Solução...,1
1,1.b - [análise e projeto] Concepção da Solução...,1
2,1.c - [análise e projeto] Concepção da Solução...,1
3,2. - [análise e projeto] Planejamento do Proj...,2
4,"3. - [análise e projeto] Levantamento, Anális...",3
...,...,...
90,37. - [apoio ao desenvolvimento] Prospecção T...,37
91,38. - [design e UX] Wireframe de Funcionalida...,38
92,39. - [design e UX] Prototipação de Funcional...,39
93,40. - [design e UX] Elaboração de Imagem - Co...,40


In [494]:
tarefas_df[401]

,id_tarefas,project,tracker,status,priority,author,assigned_to,fixed_version,parent,subject,description,start_date,due_date,done_ratio,is_private,estimated_hours,custom_fields,created_on_tarefas,updated_on_tarefas,closed_on,data_carga,author_id,author_name,Tipo,catalogo_redmine,issue,assigned_to_id,assigned_to_name,id_tempo_lancado,HET (Real),project_id,dias_desde_criacao,dias_desde_atualizacao,dias_desde_inicio,dias_ate_prazo,qtd_demandas
0,107742,Arquitetura para Aplicações Python,"{'id': 40, 'name': 'Atividade (STI - Memora)'}",Pronta p/ revisão do preposto,"{'id': 2, 'name': 'Média'}","{'id': 401, 'name': 'Fernando de Araujo Moraes...","{'id': 401, 'name': 'Fernando de Araujo Moraes...","{'id': 3838, 'name': 'N/A'}",102022,Elaborar a ATA da Reunião de Entrega de Ativid...,Elaborar a ATA da Reunião de Entrega de Ativid...,2022-02-17,2022-02-18,100,False,1.00,"[{'id': 119, 'name': 'Serviço (Memora)', 'valu...",2022-02-17T19:50:56Z,2022-02-18T13:25:17Z,2022-02-18 13:25:17+00:00,2022-02-18 11:13:33.153562,401,Fernando de Araujo Moraes Rodarte,Atividade (STI - Memora),36. - [apoio ao desenvolvimento] Participação...,107742,401,Fernando de Araujo Moraes Rodarte,77281,1.00,768,0,-1,1,-1,1
1,107741,Arquitetura para Aplicações Python,"{'id': 40, 'name': 'Atividade (STI - Memora)'}",Pronta p/ revisão do preposto,"{'id': 2, 'name': 'Média'}","{'id': 401, 'name': 'Fernando de Araujo Moraes...","{'id': 401, 'name': 'Fernando de Araujo Moraes...","{'id': 3838, 'name': 'N/A'}",102022,Reunião de Entrega de Atividades,Reunião de Entrega de Atividades,2022-02-17,2022-02-17,100,False,1.00,"[{'id': 119, 'name': 'Serviço (Memora)', 'valu...",2022-02-17T19:49:10Z,2022-02-17T20:05:21Z,2022-02-17 20:05:21+00:00,2022-02-18 11:13:33.153562,401,Fernando de Araujo Moraes Rodarte,Atividade (STI - Memora),36. - [apoio ao desenvolvimento] Participação...,107741,401,Fernando de Araujo Moraes Rodarte,77212,1.00,768,0,0,1,-2,1
2,107714,LIGO - Laboratório de Inovação Goiás,"{'id': 40, 'name': 'Atividade (STI - Memora)'}",Pronta p/ revisão do preposto,"{'id': 2, 'name': 'Média'}","{'id': 401, 'name': 'Fernando de Araujo Moraes...","{'id': 401, 'name': 'Fernando de Araujo Moraes...","{'id': 3838, 'name': 'N/A'}",106253,Participação da Reunião Ordinária do Comitê de...,Participação da Reunião Ordinária do Comitê de...,2022-02-17,2022-02-17,100,False,1.00,"[{'id': 119, 'name': 'Serviço (Memora)', 'valu...",2022-02-17T17:38:06Z,2022-02-17T17:46:12Z,2022-02-17 17:46:12+00:00,2022-02-18 11:13:33.153562,401,Fernando de Araujo Moraes Rodarte,Atividade (STI - Memora),36. - [apoio ao desenvolvimento] Participação...,107714,401,Fernando de Araujo Moraes Rodarte,77190,1.00,489,0,0,1,-2,1
3,107408,TR BALANCAS GOINFRA + ECONOMIA,"{'id': 40, 'name': 'Atividade (STI - Memora)'}",Pronta p/ revisão do preposto,"{'id': 2, 'name': 'Média'}","{'id': 401, 'name': 'Fernando de Araujo Moraes...","{'id': 401, 'name': 'Fernando de Araujo Moraes...","{'id': 3838, 'name': 'N/A'}",105636,Elaborar a ATA da Reunião de Entrega de Ativid...,Elaborar a ATA da Reunião de Entrega de Ativid...,2022-02-15,2022-02-15,100,False,1.00,"[{'id': 119, 'name': 'Serviço (Memora)', 'valu...",2022-02-15T16:30:36Z,2022-02-15T19:04:39Z,2022-02-15 19:04:39+00:00,2022-02-18 11:13:33.153562,401,Fernando de Araujo Moraes Rodarte,Atividade (STI - Memora),36. - [apoio ao desenvolvimento] Participação...,107408,401,Fernando de Araujo Moraes Rodarte,76887,1.00,788,2,2,3,-4,1
4,107407,TR BALANCAS GOINFRA + ECONOMIA,"{'id': 40, 'name': 'Atividade (STI - Memora)'}",Pronta p/ revisão do preposto,"{'id': 2, 'name': 'Média'}","{'id': 401, 'name': 'Fernando de Araujo Moraes...","{'id': 401, 'name': 'Fernando de Araujo Moraes...","{'id': 3838, 'name': 'N/A'}",105636,Reunião de Entrega de Atividades e Acompanhame...,Reunião de Entrega de Atividades e Acompanhame...,2022-02-15,2022-02-15,100,False,1.00,"[{'id': 119, 'name': 'Serviço (Memora)', 'valu...",2022-02-15T16:27:42Z,2022-02-15T16:43:46Z,2022-02-15 16:43:46+00:00,2022-

In [495]:
# Merge tarefas com o catalogo do redmine
for colaborador in tarefas_df:
    tarefas_df[colaborador] = pd.merge(
        tarefas_df[colaborador], 
        gestao_simplificada['catalogo_redmine'], 
        how="inner", 
        left_on="catalogo_redmine", 
        right_on="Catalogo_redmine"
    )

In [496]:
# Merge tarefas com o catalogo SEDI
for colaborador in tarefas_df:
    tarefas_df[colaborador] = pd.merge(
        tarefas_df[colaborador], 
        gestao_simplificada['catalogo'],
        how="inner",
        on="Codg_relacionamento"
    )

In [497]:
# Merge com a tabela de colaboradores para pegar as informações de 'Perfil Real' e 'Fator Complexidade Real''
for colaborador in tarefas_df:
    tarefas_df[colaborador] = pd.merge(
        tarefas_df[colaborador],
        gestao_simplificada['colaboradores'][['id_redmine', 'Perfil Real', 'Fator Complexidade Real']],
        how="inner",
        left_on="author_id",
        right_on="id_redmine"
    )

In [498]:
#tarefas_df[401]['created_on_tarefas'][0]
# Converte as string em datetime
#from datetime import timezone
#data = datetime.fromisoformat(tarefas_df[401]['created_on_tarefas'][0][:-1]).astimezone(timezone.utc)
#print(data)

In [499]:
# Renomeia as colunas de coplexidade
gestao_simplificada['complexidade'] = gestao_simplificada['complexidade'].rename(columns={'Sistema-Modulo':'sistema_modulo',
                                 'Complexidade-Modulo':'complexidade_modulo'})

gestao_simplificada["complexidade"] = gestao_simplificada["complexidade"][['complexidade_modulo', 'sistema_modulo']]

In [500]:
valor_ust = 51.50;

In [501]:
tarefas_df[401]

,id_tarefas,project,tracker,status,priority,author,assigned_to,fixed_version,parent,subject,description,start_date,due_date,done_ratio,is_private,estimated_hours,custom_fields,created_on_tarefas,updated_on_tarefas,closed_on,data_carga,author_id,author_name,Tipo,catalogo_redmine,issue,assigned_to_id,assigned_to_name,id_tempo_lancado,HET (Real),project_id,dias_desde_criacao,dias_desde_atualizacao,dias_desde_inicio,dias_ate_prazo,qtd_demandas,Catalogo_redmine,Codg_relacionamento,nr_servico,Grupo de Despesa,Serviço,ID,Complexidade,Escopo,HET (máx),Entregáveis,Perfis Exigidos,Classe,Grupo de Serviços,Grupo de Serviços Ajustado,Atividades Desempenhadas,id_redmine,Perfil Real,Fator Complexidade Real
0,107742,Arquitetura para Aplicações Python,"{'id': 40, 'name': 'Atividade (STI - Memora)'}",Pronta p/ revisão do preposto,"{'id': 2, 'name': 'Média'}","{'id': 401, 'name': 'Fernando de Araujo Moraes...","{'id': 401, 'name': 'Fernando de Araujo Moraes...","{'id': 3838, 'name': 'N/A'}",102022,Elaborar a ATA da Reunião de Entrega de Ativid...,Elaborar a ATA da Reunião de Entrega de Ativid...,2022-02-17,2022-02-18,100,False,1.00,"[{'id': 119, 'name': 'Serviço (Memora)', 'valu...",2022-02-17T19:50:56Z,2022-02-18T13:25:17Z,2022-02-18 13:25:17+00:00,2022-02-18 11:13:33.153562,401,Fernando de Araujo Moraes Rodarte,Atividade (STI - Memora),36. - [apoio ao desenvolvimento] Participação...,107742,401,Fernando de Araujo Moraes Rodarte,77281,1.00,768,0,-1,1,-1,1,36. - [apoio ao desenvolvimento] Participação...,36,36,DESPESA CORRENTE,Participação em Reuniões,-,Única,"Por Reunião de, no mínimo, 60 minutos.",1,Atas de Reunião,Todos,Gerência de Configuração,Todos os Grupos de Serviços,Todos os Grupos de Serviços,Participação em Reuniões não contempladas na e...,401,Gerente de Projeto Sênior,4.20
1,107741,Arquitetura para Aplicações Python,"{'id': 40, 'name': 'Atividade (STI - Memora)'}",Pronta p/ revisão do preposto,"{'id': 2, 'name': 'Média'}","{'id': 401, 'name': 'Fernando de Araujo Moraes...","{'id': 401, 'name': 'Fernando de Araujo Moraes...","{'id': 3838, 'name': 'N/A'}",102022,Reunião de Entrega de Atividades,Reunião de Entrega de Atividades,2022-02-17,2022-02-17,100,False,1.00,"[{'id': 119, 'name': 'Serviço (Memora)', 'valu...",2022-02-17T19:49:10Z,2022-02-17T20:05:21Z,2022-02-17 20:05:21+00:00,2022-02-18 11:13:33.153562,401,Fernando de Araujo Moraes Rodarte,Atividade (STI - Memora),36. - [apoio ao desenvolvimento] Participação...,107741,401,Fernando de Araujo Moraes Rodarte,77212,1.00,768,0,0,1,-2,1,36. - [apoio ao desenvolvimento] Participação...,36,36,DESPESA CORRENTE,Participação em Reuniões,-,Única,"Por Reunião de, no mínimo, 60 minutos.",1,Atas de Reunião,Todos,Gerência de Configuração,Todos os Grupos de Serviços,Todos os Grupos de Serviços,Participação em Reuniões não contempladas na e...,401,Gerente de Projeto Sênior,4.20
2,107714,LIGO - Laboratório de Inovação Goiás,"{'id': 40, 'name': 'Atividade (STI - Memora)'}",Pronta p/ revisão do preposto,"{'id': 2, 'name': 'Média'}","{'id': 401, 'name': 'Fernando de Araujo Moraes...","{'id': 401, 'name': 'Fernando de Araujo Moraes...","{'id': 3838, 'name': 'N/A'}",106253,Participação da Reunião Ordinária do Comitê de...,Participação da Reunião Ordinária do Comitê de...,2022-02-17,2022-02-17,100,False,1.00,"[{'id': 119, 'name': 'Serviço (Memora)', 'valu...",2022-02-17T17:38:06Z,2022-02-17T17:46:12Z,2022-02-17 17:46:12+00:00,2022-02-18 11:13:33.153562,401,Fernando de Araujo Moraes Rodarte,Atividade (STI - Memora),36. - [apoio ao desenvolvimento] Participação...,107714,401,Fernando de Araujo Moraes Rodarte,77190,1.00,489,0,0,1,-2,1,36. - [apoio ao desenvolvimento] Participação...,36,36,DESPESA CORRENTE,Participação em Reuniões,-,Única,"Por Reunião de, no mínimo, 60 minutos.",1,Atas de Reunião,Todos,Gerência de Configuração,Todos os Grupos de Serviços,Todos os Grupos de Serviços,Participação em Reuniões não contempladas na e...,401,Gerente de Projeto Sênior,4.20
3,107408,TR BALANCAS GOINFRA + ECONOMIA,"

In [502]:
for colaborador in tarefas_df:
    indx = tarefas_df[colaborador]['Perfis Exigidos'].index.to_list();
    tarefas_df[colaborador].loc[indx, 'Fator de complexidade'] = tarefas_df[colaborador].loc[indx, 'Fator Complexidade Real']
    tarefas_df[colaborador].loc[indx, 'Perfis Exigidos'] = tarefas_df[colaborador].loc[indx, 'Perfil Real']

In [503]:
tarefas_df[401][
    ['id_tempo_lancado',
     'issue',
     'status',
     'HET (Real)', 
     'Fator de complexidade', 
     'Perfis Exigidos',
     'estimated_hours',
     'HET (máx)']
]

,id_tempo_lancado,issue,status,HET (Real),Fator de complexidade,Perfis Exigidos,estimated_hours,HET (máx)
0,77281,107742,Pronta p/ revisão do preposto,1.00,4.20,Gerente de Projeto Sênior,1.00,1
1,77212,107741,Pronta p/ revisão do preposto,1.00,4.20,Gerente de Projeto Sênior,1.00,1
2,77190,107714,Pronta p/ revisão do preposto,1.00,4.20,Gerente de Projeto Sênior,1.00,1
3,76887,107408,Pronta p/ revisão do preposto,1.00,4.20,Gerente de Projeto Sênior,1.00,1
4,76852,107407,Pronta p/ revisão do preposto,0.50,4.20,Gerente de Projeto Sênior,1.00,1
...,...,...,...,...,...,...,...,...
69,74735,105830,Pronta p/ revisão do preposto,1.00,4.20,Gerente de Projeto Sênior,10.00,10
70,74731,105825,Pronta p/ revisão do preposto,2.00,4.20,Gerente de Projeto Sênior,10.00,10
71,74641,105713,Pronta p/ revisão do preposto,0.15,4.20,Gerente de Projeto Sênior,10.00,10
72,230464,105985,Em andamento,10.20,4.20,Gerente de Projeto Sênior,28.00,28


In [504]:
def get_qtd_servicos(campos):
    qtd=['0']
    qtd=[quantidade['value'] for quantidade in campos if quantidade['id']==64]
    if qtd==['0'] or qtd==[''] or len(qtd) == 0:
        return 1
    return qtd[0]


for colaborador in tarefas_df:
    tarefas_df[colaborador]['Diff_complex'] = (
        tarefas_df[colaborador]['Fator Complexidade Real'] /
        tarefas_df[colaborador]['Fator de complexidade']
    )
    
    tarefas_df[colaborador].sort_values(['id_tarefas', 'assigned_to_id'], inplace=True)
    tarefas_df[colaborador]['Repetida'] = tarefas_df[colaborador][['id_tarefas', 'assigned_to_id']].duplicated(keep=False)
    #drop_indx = list(tarefas_df[colaborador][(tarefas_df[colaborador]['Diff_complex'] < 1) & (tarefas_df[colaborador]['Repetida'])])
    #tarefas_df[colaborador].drop(labels=drop_indx, axis=0, inplace=True)
    
    tarefas_df[colaborador]['qtd_servicos'] = 1;
    
    tarefas_df[colaborador]['Escopo'].fillna('-', inplace=True)
    tarefas_df[colaborador]['Excedeu o HET (MAX)'] = (
        tarefas_df[colaborador]['HET (Real)'] >
        (tarefas_df[colaborador]['qtd_servicos'].multiply(tarefas_df[colaborador]['HET (máx)']))
    )

In [505]:
tarefas_df[401]

,id_tarefas,project,tracker,status,priority,author,assigned_to,fixed_version,parent,subject,description,start_date,due_date,done_ratio,is_private,estimated_hours,custom_fields,created_on_tarefas,updated_on_tarefas,closed_on,data_carga,author_id,author_name,Tipo,catalogo_redmine,issue,assigned_to_id,assigned_to_name,id_tempo_lancado,HET (Real),project_id,dias_desde_criacao,dias_desde_atualizacao,dias_desde_inicio,dias_ate_prazo,qtd_demandas,Catalogo_redmine,Codg_relacionamento,nr_servico,Grupo de Despesa,Serviço,ID,Complexidade,Escopo,HET (máx),Entregáveis,Perfis Exigidos,Classe,Grupo de Serviços,Grupo de Serviços Ajustado,Atividades Desempenhadas,id_redmine,Perfil Real,Fator Complexidade Real,Fator de complexidade,Diff_complex,Repetida,qtd_servicos,Excedeu o HET (MAX)
54,105619,LIGO - Laboratório de Inovação Goiás,"{'id': 40, 'name': 'Atividade (STI - Memora)'}",Cancelada,"{'id': 2, 'name': 'Média'}","{'id': 401, 'name': 'Fernando de Araujo Moraes...","{'id': 401, 'name': 'Fernando de Araujo Moraes...","{'id': 3838, 'name': 'N/A'}",105385,Integração com a Equipe da Memora,Integração com a Equipe da Memora,2022-01-20,2022-01-20,100,False,3.00,"[{'id': 119, 'name': 'Serviço (Memora)', 'valu...",2022-01-26T18:42:29Z,2022-02-11T11:30:24Z,2022-01-26 18:57:00+00:00,2022-02-18 11:13:33.153562,401,Fernando de Araujo Moraes Rodarte,Atividade (STI - Memora),36. - [apoio ao desenvolvimento] Participação...,105619,401,Fernando de Araujo Moraes Rodarte,74530,3.00,489,22,6,29,-30,1,36. - [apoio ao desenvolvimento] Participação...,36,36,DESPESA CORRENTE,Participação em Reuniões,-,Única,"Por Reunião de, no mínimo, 60 minutos.",1,Atas de Reunião,Gerente de Projeto Sênior,Gerência de Configuração,Todos os Grupos de Serviços,Todos os Grupos de Serviços,Participação em Reuniões não contempladas na e...,401,Gerente de Projeto Sênior,4.20,4.20,1.00,False,1,True
53,105629,LIGO - Laboratório de Inovação Goiás,"{'id': 40, 'name': 'Atividade (STI - Memora)'}",Pronta p/ revisão do preposto,"{'id': 2, 'name': 'Média'}","{'id': 401, 'name': 'Fernando de Araujo Moraes...","{'id': 401, 'name': 'Fernando de Araujo Moraes...","{'id': 3838, 'name': 'N/A'}",105385,Reunião de Alinhamento para Ínício de Trabalho...,Reunião com o Gerente de Projetos Paulo Robert...,2022-01-20,2022-01-20,100,False,1.00,"[{'id': 119, 'name': 'Serviço (Memora)', 'valu...",2022-01-26T19:39:52Z,2022-02-08T18:15:10Z,2022-01-26 19:54:24+00:00,2022-02-18 11:13:33.153562,401,Fernando de Araujo Moraes Rodarte,Atividade (STI - Memora),36. - [apoio ao desenvolvimento] Participação...,105629,401,Fernando de Araujo Moraes Rodarte,74541,1.00,489,22,9,29,-30,1,36. - [apoio ao desenvolvimento] Participação...,36,36,DESPESA CORRENTE,Participação em Reuniões,-,Única,"Por Reunião de, no mínimo, 60 minutos.",1,Atas de Reunião,Gerente de Projeto Sênior,Gerência de Configuração,Todos os Grupos de Serviços,Todos os Grupos de Serviços,Participação em Reuniões não contempladas na e...,401,Gerente de Projeto Sênior,4.20,4.20,1.00,False,1,False
52,105632,LIGO - Laboratório de Inovação Goiás,"{'id': 40, 'name': 'Atividade (STI - Memora)'}",Pronta p/ revisão do preposto,"{'id': 2, 'name': 'Média'}","{'id': 401, 'name': 'Fernando de Araujo Moraes...","{'id': 401, 'name': 'Fernando de Araujo Moraes...","{'id': 3838, 'name': 'N/A'}",105385,Reunião Presencial de Integração c/ a Equipe d...,Reunião Presencial de Integração c/ a Equipe d...,2022-01-21,2022-01-21,100,False,1.00,"[{'id': 119, 'name': 'Serviço (Memora)', 'valu...",2022-01-26T20:45:22Z,2022-02-08T18:15:10Z,2022-01-26 21:12:37+00:00,2022-02-18 11:13:33.153562,401,Fernando de Araujo Moraes Rodarte,Atividade (STI - Memora),36. - [apoio ao desenvolvimento] Participação...,105632,401,Fernando de Araujo Moraes Rodarte,74545,1.00,489,22,9,28,-29,1,36. - [apoio ao desenvolvimento] Participação...,36,36,DESPESA CORRENTE,Participação em Reuniões,-,Única,"Por Reunião de, no mínimo, 60 minutos.",1,Atas de Reunião,Gerente de Projeto Sênior,Gerência de Configuração,T

In [506]:
# gera os novos campos de analise das atividades
for colaborador in tarefas_df: 
    tarefas_df[colaborador]['UST Real'] = (
        tarefas_df[colaborador]['HET (Real)'] * tarefas_df[colaborador]['Fator de complexidade']
    )
    
    tarefas_df[colaborador]['UST Real Ajustado'] = (
        [
            row['HET (máx)'] * row['Fator de complexidade'] * row['HET (Real)']
            if row['Excedeu o HET (MAX)'] else
            row['HET (Real)'] * row['Fator de complexidade']
            for index, row in tarefas_df[colaborador].iterrows()
        ]
    )
    
    tarefas_df[colaborador]['UST Previsto'] = (
        tarefas_df[colaborador]['estimated_hours'] * tarefas_df[colaborador]['Fator de complexidade']
    )
         
    tarefas_df[colaborador]['HET perdido'] = (
        [
            row['HET (máx)'] * row['qtd_servicos'] - row['HET (Real)']
            if (row['HET (máx)'] * row['qtd_servicos'] - row['HET (Real)']) > 0 else 0
            for index, row in tarefas_df[colaborador].iterrows()
        ]
    )
    
    tarefas_df[colaborador]['Faturamento perdido'] = (
        tarefas_df[colaborador]['HET perdido'].multiply(
            tarefas_df[colaborador]['Fator de complexidade']
        )
    ) * valor_ust
    
    tarefas_df[colaborador]['HET (máx)'] = (
        tarefas_df[colaborador]['qtd_servicos'].multiply(
            tarefas_df[colaborador]['HET (máx)']
        )
    )
    
    tarefas_df[colaborador]['por_hora'] = (
        (
            tarefas_df[colaborador]['Escopo'].str.contains('Por hora') | 
            tarefas_df[colaborador]['Escopo'].str.contains('Por Reunião')
        )
    )
    
    tarefas_df[colaborador]['HET (máx)'] = (
        [
            row['HET (Real)'] if row['por_hora'] else row['HET (máx)'] for index, row in tarefas_df[colaborador].iterrows()
        ]
    )
    
    tarefas_df[colaborador]['A Faturar Real'] = (
        tarefas_df[colaborador]['UST Real'] * valor_ust
    )
    
    tarefas_df[colaborador]['A Faturar Real Ajustado'] = (
        tarefas_df[colaborador]['UST Real Ajustado'] * valor_ust
    )
    
    tarefas_df[colaborador]['A Faturar Previsto'] = (
        tarefas_df[colaborador]['UST Previsto'] * valor_ust
    )

In [507]:
for colaborador in tarefas_df:
    tarefas_df[colaborador] = tarefas_df[colaborador].rename(columns={'closed_on':'data_conclusao',
                                             'created_on_tarefas':'data_criacao',
                                             'description':'descricao',
                                             'done_ratio':'percentual_concluido',
                                             'due_date':'data_prevista',
                                             'estimated_hours':'estimated_hours',
                                             'id':'#_redmine',
                                             'parent':'tarefa_pai',
                                             'project':'Projeto',
                                             'start_date': 'data_inicio',
                                             'estimated_hours': 'HET_previsto',
                                             'Catalogo_redmine': 'Catálogo_01 (Memora - 13/2020)',
                                             'HET (Real)':'HET_real',
                                             'subject':'titulo',
                                             'updated_on_tarefas':'data_atualizacao',
                                             'assigned_to_id':'id_atribuido_para',
                                             'assigned_to_name':'atribuido_para',
                                             'author_id':'id_autor',
                                             'A Faturar':'A Faturar Real',
                                             'UST':'UST Real',
                                             'Situaçăo':'status',
                                             'author_name':'autor'})

In [508]:
for colaborador in tarefas_df:
    
    try:
        
        for index, valor in enumerate(tarefas_df[colaborador]['data_criacao']):
            
            if type(valor)  == pd._libs.tslibs.timestamps.Timestamp:
                tarefas_df[colaborador].at[index, 'data_criacao'] = valor.strftime('%d-%m-%Y %H:%M:%S')      
            else: 
                if type(valor) == pd._libs.tslibs.nattype.NaTType or valor is None:
                        tarefas_df[colaborador].at[index, 'data_criacao'] = None;
                else:
                    tarefas_df[colaborador].at[index, 'data_criacao'] = (
                        pd.to_datetime(pd.to_datetime(valor)).replace(tzinfo=None).strftime('%d-%m-%Y %H:%M:%S')

                    )
            
        # Normaliza a data de atualizacao
        for index, valor in enumerate(tarefas_df[colaborador]['data_atualizacao']):
            
            if type(valor)  == pd._libs.tslibs.timestamps.Timestamp:
                tarefas_df[colaborador].at[index, 'data_atualizacao'] = valor.strftime('%d-%m-%Y %H:%M:%S')
            else:
                if type(valor) == pd._libs.tslibs.nattype.NaTType or valor is None:
                        tarefas_df[colaborador].at[index, 'data_atualizacao'] = None;
                else:
                    tarefas_df[colaborador].at[index, 'data_atualizacao'] = (
                        pd.to_datetime(pd.to_datetime(valor)).replace(tzinfo=None).strftime('%d-%m-%Y %H:%M:%S')

                    )

        #for index, valor in enumerate(tarefas_df[colaborador]['data_conclusao']):
            

            #if type(valor)  == pd._libs.tslibs.timestamps.Timestamp:
                #tarefas_df[colaborador].at[index, 'data_conclusao'] = valor.strftime('%d-%m-%Y %H:%M:%S')
            #else:
                #if type(valor) == pd._libs.tslibs.nattype.NaTType or valor is None:
                        #tarefas_df[colaborador].at[index, 'data_conclusao'] = None;
                #else:
                    #tarefas_df[colaborador].at[index, 'data_conclusao'] = (
                        #pd.to_datetime(valor).replace(tzinfo=None).strftime('%d-%m-%Y %H:%M:%S')

                    #)
                
        for index, valor in enumerate(tarefas_df[colaborador]['data_carga']):
            if type(valor)  == pd._libs.tslibs.timestamps.Timestamp:
                tarefas_df[colaborador].at[index, 'data_carga'] = valor.strftime('%d-%m-%Y %H:%M:%S')
            else:
                if type(valor) == pd._libs.tslibs.nattype.NaTType or valor is None:
                        tarefas_df[colaborador].at[index, 'data_carga'] = None;
                else:
                    tarefas_df[colaborador].at[index, 'data_carga'] = str(valor.strftime('%d-%m-%Y %H:%M:%S'))
        
        
        for index, valor in enumerate(tarefas_df[colaborador]['data_conclusao']):
            if type(valor)  == pd._libs.tslibs.timestamps.Timestamp:
                tarefas_df[colaborador].at[index, 'data_conclusao'] = valor.strftime('%d-%m-%Y %H:%M:%S')
            else:
                if type(valor) == pd._libs.tslibs.nattype.NaTType or valor is None:
                        tarefas_df[colaborador].at[index, 'data_conclusao'] = None;
                else:
                    tarefas_df[colaborador].at[index, 'data_conclusao'] = str(valor.strftime('%d-%m-%Y %H:%M:%S'))
                
            
    except Exception as e:
        print(e)

In [509]:
def date_to_str(arg):
    if type(arg) == str:
        return arg
    if type(arg) == pd._libs.tslibs.timestamps.Timestamp:
        arg = arg.replace(tzinfo=None).strftime('%d-%m-%Y %H:%M:%S')
        return arg
    else:
        return None

# Normaliza data de conclusão
for colaborador in tarefas_df:
    tarefas_df[colaborador]['data_conclusao'] = (
        tarefas_df[colaborador]['data_conclusao'].apply(lambda data: date_to_str(data))
    )        
    
# Normaliza data da carga
for colaborador in tarefas_df:
    tarefas_df[colaborador]['data_carga'] = (
        tarefas_df[colaborador]['data_carga'].apply(lambda data: date_to_str(data))
    )        
    

In [525]:
dfs = [tarefas_df[df] for df in tarefas_df]

In [535]:
relatorio = pd.concat(dfs)

In [538]:
with pd.ExcelWriter('/home/gustavo/workspace/SEFAZ/memora/automacao/script/extracao/IndRedmineCalc.xlsx', engine='xlsxwriter') as writer:
        #for colaborador in tarefas_df:
            #query = f"id_redmine == {colaborador}"
            #print(query)
            #consulta_colaborador = gestao_simplificada['colaboradores'].query(query)["Nome Redmine"].values[0]
            #print(consulta_colaborador)
            #sheet_name = consulta_colaborador if consulta_colaborador else colaborador
            #tarefas_df[colaborador].to_excel(writer, sheet_name=sheet_name[0:30])
    sheet_name = f'relatório_{datetime.now().strftime("%d-%m-%Y %H.%M.%S")}'
    relatorio.to_excel(writer, sheet_name=sheet_name[0:30])
    writer.save()

/home/gustavo/workspace/SEFAZ/memora/automacao/venv/lib/python3.9/site-packages/xlsxwriter/workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")
